# Prédiction de Consommation Énergétique - Campus Aix

## Objectifs du projet

Ce notebook présente un pipeline complet de prédiction de consommation énergétique horaire pour un campus, en utilisant :
- Des données temporelles (heure, jour, mois, année)
- Des variables météorologiques
- Des features dérivées (lags, différences, encodages cycliques)
- Un modèle de réseau de neurones (MLPRegressor)

**Objectif métier** : Prédire la consommation énergétique horaire pour optimiser la gestion des ressources.


## 1. Imports et Configuration

Importation des bibliothèques nécessaires et configuration de la reproductibilité.


In [ ]:
import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Configuration pour la reproductibilité
np.random.seed(42)

# Configuration des graphiques
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("✓ Imports effectués avec succès")


## 2. Chargement des Données

Chargement du fichier CSV contenant les données de consommation et les variables explicatives.


In [ ]:
# Chargement des données
data = np.genfromtxt('conso_campus_aix.csv', delimiter=',', skip_header=1)

# Extraction de la variable cible (colonne 8 : consommation)
conso = data[:, 8]
y_orig = conso

# Extraction des variables temporelles
Jours_du_mois = data[:, 0].astype(int)
Mois_orig = data[:, 1].astype(int)
Annees_orig = data[:, 2].astype(int)
Heure_orig = data[:, 3].astype(int)

# Variables météorologiques (colonnes 4, 5, 6, 7)
variables_meteo = data[:, [4, 5, 6, 7]]

print(f"✓ Données chargées : {len(data)} observations")
print(f"  - Consommation : min={y_orig.min():.2f}, max={y_orig.max():.2f}, mean={y_orig.mean():.2f}")
print(f"  - Période : {Annees_orig.min()}-{Mois_orig.min():02d} à {Annees_orig.max()}-{Mois_orig.max():02d}")


## 3. Exploration des Données (EDA)

### 3.1. Analyse de la variable cible

Visualisation de la distribution et de l'évolution temporelle de la consommation pour identifier :
- Les tendances et saisonnalités
- Les valeurs aberrantes potentielles
- La distribution des valeurs


In [ ]:
# Visualisation de l'évolution temporelle de la consommation
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Série temporelle complète
axes[0, 0].plot(y_orig[:1000], alpha=0.7, linewidth=0.5)
axes[0, 0].set_title('Évolution de la consommation (1000 premières heures)')
axes[0, 0].set_xlabel('Heures')
axes[0, 0].set_ylabel('Consommation')
axes[0, 0].grid(True, alpha=0.3)

# Distribution
axes[0, 1].hist(y_orig, bins=50, edgecolor='black', alpha=0.7)
axes[0, 1].set_title('Distribution de la consommation')
axes[0, 1].set_xlabel('Consommation')
axes[0, 1].set_ylabel('Fréquence')
axes[0, 1].axvline(y_orig.mean(), color='r', linestyle='--', label=f'Moyenne: {y_orig.mean():.2f}')
axes[0, 1].legend()

# Box plot pour détecter les outliers
axes[1, 0].boxplot(y_orig, vert=True)
axes[1, 0].set_title('Box Plot - Détection des valeurs aberrantes')
axes[1, 0].set_ylabel('Consommation')
axes[1, 0].grid(True, alpha=0.3)

# Statistiques descriptives
stats_text = f"""
Statistiques descriptives :
- Min: {y_orig.min():.2f}
- Max: {y_orig.max():.2f}
- Moyenne: {y_orig.mean():.2f}
- Médiane: {np.median(y_orig):.2f}
- Écart-type: {y_orig.std():.2f}
- Skewness: {np.abs(y_orig - y_orig.mean()).mean() / y_orig.std():.2f}
"""
axes[1, 1].text(0.1, 0.5, stats_text, fontsize=11, verticalalignment='center',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()


### 3.2. Analyse des patterns temporels

Analyse de la consommation selon l'heure, le jour de la semaine et le mois pour identifier les patterns cycliques.


In [ ]:
# Calcul des jours de la semaine pour l'analyse
jours_de_la_semaine_temp = []
for annee, mois, jour in zip(Annees_orig, Mois_orig, Jours_du_mois):
    date_obj = datetime.date(annee, mois, jour)
    jours_de_la_semaine_temp.append(date_obj.weekday())
jours_de_la_semaine_temp = np.array(jours_de_la_semaine_temp)

# Visualisation des patterns temporels
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Consommation par heure
conso_par_heure = [y_orig[Heure_orig == h].mean() for h in range(24)]
axes[0, 0].plot(range(24), conso_par_heure, marker='o', linewidth=2, markersize=6)
axes[0, 0].set_title('Consommation moyenne par heure de la journée')
axes[0, 0].set_xlabel('Heure')
axes[0, 0].set_ylabel('Consommation moyenne')
axes[0, 0].set_xticks(range(0, 24, 2))
axes[0, 0].grid(True, alpha=0.3)

# Consommation par jour de la semaine
jours_noms = ['Lun', 'Mar', 'Mer', 'Jeu', 'Ven', 'Sam', 'Dim']
conso_par_jour = [y_orig[jours_de_la_semaine_temp == j].mean() for j in range(7)]
axes[0, 1].bar(jours_noms, conso_par_jour, color='steelblue', alpha=0.7)
axes[0, 1].set_title('Consommation moyenne par jour de la semaine')
axes[0, 1].set_ylabel('Consommation moyenne')
axes[0, 1].grid(True, alpha=0.3, axis='y')

# Consommation par mois
conso_par_mois = [y_orig[Mois_orig == m].mean() for m in range(1, 13)]
mois_noms = ['Jan', 'Fév', 'Mar', 'Avr', 'Mai', 'Jun', 
             'Jul', 'Aoû', 'Sep', 'Oct', 'Nov', 'Déc']
axes[1, 0].plot(range(1, 13), conso_par_mois, marker='o', linewidth=2, markersize=6, color='green')
axes[1, 0].set_title('Consommation moyenne par mois')
axes[1, 0].set_xlabel('Mois')
axes[1, 0].set_ylabel('Consommation moyenne')
axes[1, 0].set_xticks(range(1, 13))
axes[1, 0].set_xticklabels(mois_noms, rotation=45)
axes[1, 0].grid(True, alpha=0.3)

# Heatmap consommation par heure et jour de la semaine
heatmap_data = np.zeros((7, 24))
for j in range(7):
    for h in range(24):
        mask = (jours_de_la_semaine_temp == j) & (Heure_orig == h)
        if mask.sum() > 0:
            heatmap_data[j, h] = y_orig[mask].mean()
im = axes[1, 1].imshow(heatmap_data, aspect='auto', cmap='YlOrRd', interpolation='nearest')
axes[1, 1].set_title('Heatmap : Consommation par heure et jour de la semaine')
axes[1, 1].set_xlabel('Heure')
axes[1, 1].set_ylabel('Jour de la semaine')
axes[1, 1].set_yticks(range(7))
axes[1, 1].set_yticklabels(jours_noms)
axes[1, 1].set_xticks(range(0, 24, 2))
plt.colorbar(im, ax=axes[1, 1])

plt.tight_layout()
plt.show()


### 3.3. Analyse des corrélations et valeurs manquantes

Vérification des corrélations entre variables météorologiques et la consommation, ainsi que détection des valeurs manquantes.


In [ ]:
# Vérification des valeurs manquantes
missing_data = np.isnan(data).sum(axis=0)
print("Valeurs manquantes par colonne :")
for i, missing in enumerate(missing_data):
    if missing > 0:
        print(f"  Colonne {i}: {missing} valeurs manquantes ({missing/len(data)*100:.2f}%)")
    else:
        print(f"  Colonne {i}: Aucune valeur manquante")

# Analyse des corrélations avec les variables météorologiques
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Corrélations entre variables météorologiques et consommation
corr_with_target = []
for i in range(4):
    corr = np.corrcoef(variables_meteo[:, i], y_orig)[0, 1]
    corr_with_target.append(corr)

axes[0].bar(range(1, 5), corr_with_target, color='coral', alpha=0.7)
axes[0].set_title('Corrélation entre variables météo et consommation')
axes[0].set_xlabel('Variable météo (colonne)')
axes[0].set_ylabel('Corrélation avec consommation')
axes[0].set_xticks(range(1, 5))
axes[0].axhline(0, color='black', linestyle='-', linewidth=0.5)
axes[0].grid(True, alpha=0.3, axis='y')

# Distribution des variables météorologiques
axes[1].boxplot([variables_meteo[:, i] for i in range(4)], labels=[f'Var {i+4}' for i in range(4)])
axes[1].set_title('Distribution des variables météorologiques')
axes[1].set_ylabel('Valeurs')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print(f"\n✓ Corrélations variables météo → consommation : {[f'{c:.3f}' for c in corr_with_target]}")


### 📊 Interprétation & Pistes d'amélioration - EDA

**Observations clés :**
- Distribution de la consommation : identifier si elle est normale, asymétrique, ou présente des outliers
- Patterns temporels : heures de pointe, différences week-end/semaine, saisonnalité
- Corrélations météo : quelles variables météorologiques sont les plus prédictives

**Pistes d'amélioration :**
- Appliquer une transformation (log, Box-Cox) si la distribution est asymétrique
- Créer des features d'interaction entre variables météo et variables temporelles
- Gérer les outliers si nécessaire (capping, suppression, ou modélisation séparée)


## 4. Feature Engineering

### 4.1. Encodages cycliques

Les variables temporelles (heure, mois, jour de la semaine) sont cycliques. L'encodage sinusoïdal permet au modèle de comprendre cette cyclicité (ex: 23h est proche de 0h).


In [ ]:
# Encodage cyclique de l'heure (24h)
H_sin = np.sin(2 * np.pi * Heure_orig / 24)
H_cos = np.cos(2 * np.pi * Heure_orig / 24)

# Encodage cyclique du mois (12 mois)
M_sin = np.sin(2 * np.pi * Mois_orig / 12)
M_cos = np.cos(2 * np.pi * Mois_orig / 12)

# Calcul du jour de la semaine
jours_de_la_semaine = []
for annee, mois, jour in zip(Annees_orig, Mois_orig, Jours_du_mois):
    date_obj = datetime.date(annee, mois, jour)
    jours_de_la_semaine.append(date_obj.weekday())
jours_de_la_semaine = np.array(jours_de_la_semaine)

# Encodage cyclique du jour de la semaine (7 jours)
S_sin = np.sin(2 * np.pi * jours_de_la_semaine / 7)
S_cos = np.cos(2 * np.pi * jours_de_la_semaine / 7)

print("✓ Encodages cycliques créés")
print(f"  - Heure: sin/cos (24h cycle)")
print(f"  - Mois: sin/cos (12 mois cycle)")
print(f"  - Jour semaine: sin/cos (7 jours cycle)")


### 4.2. Visualisation des encodages cycliques

Visualisation pour vérifier que les encodages capturent bien la cyclicité des variables temporelles.


In [ ]:
# Visualisation des encodages cycliques
fig, axes = plt.subplots(1, 3, figsize=(18, 4))

# Encodage heure
axes[0].scatter(H_sin[:1000], H_cos[:1000], c=Heure_orig[:1000], cmap='hsv', alpha=0.6, s=10)
axes[0].set_title('Encodage cyclique de l\'heure (sin/cos)')
axes[0].set_xlabel('sin(2π * heure / 24)')
axes[0].set_ylabel('cos(2π * heure / 24)')
axes[0].grid(True, alpha=0.3)
axes[0].set_aspect('equal')

# Encodage mois
axes[1].scatter(M_sin[:1000], M_cos[:1000], c=Mois_orig[:1000], cmap='hsv', alpha=0.6, s=10)
axes[1].set_title('Encodage cyclique du mois (sin/cos)')
axes[1].set_xlabel('sin(2π * mois / 12)')
axes[1].set_ylabel('cos(2π * mois / 12)')
axes[1].grid(True, alpha=0.3)
axes[1].set_aspect('equal')

# Encodage jour de la semaine
axes[2].scatter(S_sin[:1000], S_cos[:1000], c=jours_de_la_semaine[:1000], cmap='hsv', alpha=0.6, s=10)
axes[2].set_title('Encodage cyclique du jour de la semaine (sin/cos)')
axes[2].set_xlabel('sin(2π * jour / 7)')
axes[2].set_ylabel('cos(2π * jour / 7)')
axes[2].grid(True, alpha=0.3)
axes[2].set_aspect('equal')

plt.tight_layout()
plt.show()


### 4.3. Variables binaires et catégorielles

Création de variables binaires pour capturer des effets spécifiques (week-end, heures actives).


In [ ]:
# Variable binaire Week-end (1 si samedi ou dimanche, 0 sinon)
is_weekend = (jours_de_la_semaine >= 5).astype(int)

# Variable binaire Heures actives (1 si entre 7h et 20h, 0 sinon)
# Cette variable aide le modèle à mieux caler le "talon" de consommation nocturne
is_active_hours = ((Heure_orig >= 7) & (Heure_orig <= 20)).astype(int)

# Visualisation de l'impact de ces variables
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Impact du week-end
conso_weekend = y_orig[is_weekend == 1].mean()
conso_semaine = y_orig[is_weekend == 0].mean()
axes[0].bar(['Semaine', 'Week-end'], [conso_semaine, conso_weekend], 
            color=['steelblue', 'coral'], alpha=0.7)
axes[0].set_title('Consommation moyenne : Semaine vs Week-end')
axes[0].set_ylabel('Consommation moyenne')
axes[0].grid(True, alpha=0.3, axis='y')
diff_pct = ((conso_weekend - conso_semaine) / conso_semaine) * 100
axes[0].text(0.5, max(conso_semaine, conso_weekend) * 0.95, 
             f'Différence: {diff_pct:.1f}%', ha='center', fontsize=10,
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# Impact des heures actives
conso_active = y_orig[is_active_hours == 1].mean()
conso_inactive = y_orig[is_active_hours == 0].mean()
axes[1].bar(['Heures inactives\n(21h-6h)', 'Heures actives\n(7h-20h)'], 
            [conso_inactive, conso_active], color=['darkblue', 'orange'], alpha=0.7)
axes[1].set_title('Consommation moyenne : Heures actives vs inactives')
axes[1].set_ylabel('Consommation moyenne')
axes[1].grid(True, alpha=0.3, axis='y')
diff_pct2 = ((conso_active - conso_inactive) / conso_inactive) * 100
axes[1].text(0.5, max(conso_active, conso_inactive) * 0.95, 
             f'Différence: {diff_pct2:.1f}%', ha='center', fontsize=10,
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

print(f"✓ Variables binaires créées")
print(f"  - Week-end: {is_weekend.sum()} observations ({is_weekend.sum()/len(is_weekend)*100:.1f}%)")
print(f"  - Heures actives: {is_active_hours.sum()} observations ({is_active_hours.sum()/len(is_active_hours)*100:.1f}%)")


### 4.4. Features temporelles avancées (Lags et différences)

Les lags (valeurs passées) et les différences capturent l'inertie et les tendances de la consommation.


In [ ]:
# Création des lags (valeurs passées)
lag_1 = np.roll(y_orig, 1)   # Consommation 1h avant
lag_2 = np.roll(y_orig, 2)   # Consommation 2h avant
lag_24 = np.roll(y_orig, 24) # Consommation 24h avant (même heure hier)
lag_168 = np.roll(y_orig, 168) # Consommation 168h avant (même heure la semaine dernière)

# Création des différences (tendances)
diff_1h = lag_1 - lag_2      # Variation sur 1h
diff_24h = lag_1 - lag_24    # Variation sur 24h

# Visualisation de l'importance des lags
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Corrélation avec la cible
lags = [lag_1, lag_24, lag_168]
lag_names = ['Lag 1h', 'Lag 24h', 'Lag 168h']
correlations = [np.corrcoef(y_orig, lag)[0, 1] for lag in lags]

axes[0, 0].bar(lag_names, correlations, color='teal', alpha=0.7)
axes[0, 0].set_title('Corrélation des lags avec la consommation actuelle')
axes[0, 0].set_ylabel('Corrélation')
axes[0, 0].set_ylim([0, 1])
axes[0, 0].grid(True, alpha=0.3, axis='y')

# Visualisation des lags sur un extrait
sample_idx = slice(200, 400)
axes[0, 1].plot(y_orig[sample_idx], label='Consommation actuelle', linewidth=2)
axes[0, 1].plot(lag_1[sample_idx], label='Lag 1h', alpha=0.7, linestyle='--')
axes[0, 1].plot(lag_24[sample_idx], label='Lag 24h', alpha=0.7, linestyle=':')
axes[0, 1].set_title('Comparaison consommation actuelle vs lags (zoom 200h)')
axes[0, 1].set_xlabel('Heures')
axes[0, 1].set_ylabel('Consommation')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Visualisation des différences
axes[1, 0].plot(diff_1h[sample_idx], label='Différence 1h', alpha=0.7, linewidth=1.5)
axes[1, 0].plot(diff_24h[sample_idx], label='Différence 24h', alpha=0.7, linewidth=1.5)
axes[1, 0].axhline(0, color='black', linestyle='-', linewidth=0.5)
axes[1, 0].set_title('Variations temporelles (différences)')
axes[1, 0].set_xlabel('Heures')
axes[1, 0].set_ylabel('Différence')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Scatter plot lag_24 vs consommation (très corrélé)
axes[1, 1].scatter(lag_24[168:], y_orig[168:], alpha=0.3, s=5)
axes[1, 1].plot([y_orig.min(), y_orig.max()], [y_orig.min(), y_orig.max()], 
                'r--', lw=2, label='y=x')
axes[1, 1].set_title(f'Lag 24h vs Consommation actuelle (corr={correlations[1]:.3f})')
axes[1, 1].set_xlabel('Consommation il y a 24h')
axes[1, 1].set_ylabel('Consommation actuelle')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"✓ Features temporelles créées")
print(f"  - Lags: 1h, 2h, 24h, 168h")
print(f"  - Différences: 1h, 24h")


### 📊 Interprétation & Pistes d'amélioration - Feature Engineering

**Observations clés :**
- Les lags (surtout lag_24h) sont très corrélés avec la consommation actuelle → indicateurs puissants
- Les encodages cycliques capturent bien les patterns temporels
- Les variables binaires (week-end, heures actives) montrent des différences significatives

**Pistes d'amélioration :**
- Ajouter des lags supplémentaires (lag_48h, lag_336h pour capturer des cycles plus longs)
- Créer des features d'interaction (ex: week-end × heure, météo × saison)
- Tester des moyennes mobiles (rolling mean) pour lisser les variations
- Ajouter des features de tendance (dérivée seconde, accélération)


## 5. Préparation des Données pour le Modèle

### 5.1. Construction de la matrice de features

Assemblage de toutes les features créées en une matrice X et préparation de la variable cible y.


In [ ]:
# Construction de la matrice de features complète
X_full = np.column_stack((
    variables_meteo,           # 4 variables météorologiques
    M_sin, M_cos,              # Encodage cyclique mois (2 features)
    H_sin, H_cos,              # Encodage cyclique heure (2 features)
    S_sin, S_cos,              # Encodage cyclique jour semaine (2 features)
    is_weekend,                # Variable binaire week-end (1 feature)
    is_active_hours,           # Variable binaire heures actives (1 feature)
    lag_1, lag_24, lag_168,    # Lags temporels (3 features)
    diff_1h, diff_24h          # Différences temporelles (2 features)
))

# Suppression des premières lignes (nécessaires pour les lags)
# On garde seulement les observations où tous les lags sont disponibles
X = X_full[168:]
y = y_orig[168:]

print(f"✓ Matrice de features construite")
print(f"  - Nombre de features: {X.shape[1]}")
print(f"  - Nombre d'observations: {X.shape[0]} (après suppression des 168 premières)")
print(f"  - Features: {X.shape[1]} variables")


### 5.2. Transformation de la variable cible

Application d'une transformation logarithmique (log1p) pour stabiliser la variance et améliorer les performances du modèle sur des données potentiellement asymétriques.


In [ ]:
# Transformation logarithmique de la variable cible
y_log = np.log1p(y)  # log1p(x) = log(1+x) pour gérer les valeurs proches de 0

# Visualisation de l'impact de la transformation
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Distribution avant transformation
axes[0].hist(y, bins=50, edgecolor='black', alpha=0.7, color='steelblue')
axes[0].set_title('Distribution de y (avant transformation)')
axes[0].set_xlabel('Consommation')
axes[0].set_ylabel('Fréquence')
axes[0].axvline(y.mean(), color='r', linestyle='--', label=f'Moyenne: {y.mean():.2f}')
axes[0].legend()

# Distribution après transformation
axes[1].hist(y_log, bins=50, edgecolor='black', alpha=0.7, color='coral')
axes[1].set_title('Distribution de y_log (après log1p)')
axes[1].set_xlabel('log(1 + Consommation)')
axes[1].set_ylabel('Fréquence')
axes[1].axvline(y_log.mean(), color='r', linestyle='--', label=f'Moyenne: {y_log.mean():.2f}')
axes[1].legend()

plt.tight_layout()
plt.show()

# Comparaison des statistiques
print("Statistiques avant/après transformation :")
print(f"  Avant - Skewness: {np.abs(y - y.mean()).mean() / y.std():.3f}")
print(f"  Après - Skewness: {np.abs(y_log - y_log.mean()).mean() / y_log.std():.3f}")
print(f"  ✓ Transformation appliquée pour stabiliser la variance")


### 5.3. Split train/test séquentiel

Pour les données temporelles, on utilise un split séquentiel (pas de shuffle) pour préserver l'ordre chronologique.


In [ ]:
# Split séquentiel (80% train, 20% test)
test_size = 0.2
split_index = int(len(X) * (1 - test_size))

X_train, X_test = X[:split_index], X[split_index:]
y_train_log, y_test_log = y_log[:split_index], y_log[split_index:]
y_test_real = y[split_index:]  # On garde aussi les valeurs réelles pour l'évaluation

print(f"✓ Split séquentiel effectué")
print(f"  - Train: {len(X_train)} observations ({len(X_train)/len(X)*100:.1f}%)")
print(f"  - Test: {len(X_test)} observations ({len(X_test)/len(X)*100:.1f}%)")
print(f"  - Période train: indices 0 à {split_index-1}")
print(f"  - Période test: indices {split_index} à {len(X)-1}")

# Visualisation du split
fig, ax = plt.subplots(figsize=(15, 4))
ax.plot(range(len(y)), y, alpha=0.6, linewidth=0.5, label='Données complètes')
ax.axvline(split_index, color='r', linestyle='--', linewidth=2, label=f'Split train/test (index {split_index})')
ax.set_title('Visualisation du split train/test')
ax.set_xlabel('Index')
ax.set_ylabel('Consommation')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()


### 5.4. Normalisation des features

Standardisation des features pour que toutes les variables soient à la même échelle, ce qui améliore les performances des réseaux de neurones.


In [ ]:
# Normalisation avec StandardScaler (moyenne=0, écart-type=1)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Visualisation de l'impact de la normalisation
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Distribution d'une feature avant normalisation (ex: première variable météo)
feature_idx = 0
axes[0].hist(X_train[:, feature_idx], bins=50, edgecolor='black', alpha=0.7, color='steelblue')
axes[0].set_title(f'Feature {feature_idx} avant normalisation')
axes[0].set_xlabel('Valeur')
axes[0].set_ylabel('Fréquence')
axes[0].axvline(X_train[:, feature_idx].mean(), color='r', linestyle='--', 
                label=f'Moyenne: {X_train[:, feature_idx].mean():.2f}')
axes[0].legend()

# Distribution après normalisation
axes[1].hist(X_train_scaled[:, feature_idx], bins=50, edgecolor='black', alpha=0.7, color='coral')
axes[1].set_title(f'Feature {feature_idx} après normalisation (StandardScaler)')
axes[1].set_xlabel('Valeur normalisée')
axes[1].set_ylabel('Fréquence')
axes[1].axvline(X_train_scaled[:, feature_idx].mean(), color='r', linestyle='--', 
                label=f'Moyenne: {X_train_scaled[:, feature_idx].mean():.3f}')
axes[1].legend()

plt.tight_layout()
plt.show()

print(f"✓ Normalisation effectuée")
print(f"  - Moyenne train (après scaling): {X_train_scaled.mean():.6f} (≈ 0)")
print(f"  - Écart-type train (après scaling): {X_train_scaled.std():.6f} (≈ 1)")


## 6. Modélisation

### 6.1. Configuration et entraînement du modèle MLP

Utilisation d'un Multi-Layer Perceptron (réseau de neurones) pour capturer les relations non-linéaires complexes entre les features et la consommation.


In [ ]:
# Configuration du modèle MLPRegressor
mlp = MLPRegressor(
    hidden_layer_sizes=(100, 100, 50),  # Architecture: 3 couches cachées
    activation='relu',                   # Fonction d'activation ReLU
    solver='adam',                       # Optimiseur Adam
    max_iter=2000,                       # Nombre maximum d'itérations
    early_stopping=True,                 # Arrêt anticipé si pas d'amélioration
    n_iter_no_change=20,                # Nombre d'itérations sans amélioration avant arrêt
    alpha=0.5,                           # Régularisation L2
    random_state=42                      # Pour la reproductibilité
)

print("Configuration du modèle MLP:")
print(f"  - Architecture: {mlp.hidden_layer_sizes}")
print(f"  - Activation: {mlp.activation}")
print(f"  - Optimiseur: {mlp.solver}")
print(f"  - Régularisation (alpha): {mlp.alpha}")
print(f"  - Early stopping: {mlp.early_stopping}")

# Entraînement du modèle
print("\n⏳ Entraînement en cours...")
mlp.fit(X_train_scaled, y_train_log)
print("✓ Modèle entraîné avec succès")


### 6.2. Analyse de la courbe d'apprentissage

Visualisation de l'évolution de la loss pendant l'entraînement pour détecter le surapprentissage ou le sous-apprentissage.


In [ ]:
# Récupération de la courbe de loss
if hasattr(mlp, 'loss_curve_'):
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(mlp.loss_curve_, linewidth=2, color='steelblue')
    ax.set_title('Courbe d\'apprentissage - Évolution de la loss')
    ax.set_xlabel('Itérations')
    ax.set_ylabel('Loss (MSE)')
    ax.grid(True, alpha=0.3)
    
    # Marquer le point d'arrêt si early stopping a été utilisé
    if mlp.n_iter_ < mlp.max_iter:
        ax.axvline(mlp.n_iter_, color='r', linestyle='--', 
                   label=f'Arrêt anticipé à l\'itération {mlp.n_iter_}')
        ax.legend()
    
    plt.tight_layout()
    plt.show()
    
    print(f"✓ Nombre d'itérations effectuées: {mlp.n_iter_}")
    print(f"  - Loss finale: {mlp.loss_curve_[-1]:.6f}")
    if len(mlp.loss_curve_) > 1:
        print(f"  - Amélioration: {((mlp.loss_curve_[0] - mlp.loss_curve_[-1]) / mlp.loss_curve_[0] * 100):.2f}%")
else:
    print("⚠ Courbe de loss non disponible (peut nécessiter validation_fraction > 0)")


### 📊 Interprétation & Pistes d'amélioration - Modélisation

**Observations clés :**
- La courbe d'apprentissage montre si le modèle converge correctement
- L'early stopping permet d'éviter le surapprentissage
- L'architecture (100, 100, 50) peut être optimisée selon les performances

**Pistes d'amélioration :**
- Tester différentes architectures (plus profondes, plus larges, ou dropout)
- Ajuster les hyperparamètres (learning_rate, alpha, batch_size)
- Utiliser la validation croisée temporelle (TimeSeriesSplit) pour mieux évaluer
- Tester d'autres modèles (XGBoost, LSTM pour séries temporelles, ensemble methods)


## 7. Évaluation du Modèle

### 7.1. Prédictions et inversion de la transformation

Les prédictions sont faites sur l'échelle logarithmique, puis inversées pour revenir à l'échelle originale.


In [ ]:
# Prédictions sur l'échelle logarithmique
y_pred_test_log = mlp.predict(X_test_scaled)

# Inversion de la transformation logarithmique
y_pred_test = np.expm1(y_pred_test_log)  # expm1(x) = exp(x) - 1, inverse de log1p

print("✓ Prédictions générées et transformées")
print(f"  - Nombre de prédictions: {len(y_pred_test)}")
print(f"  - Min prédit: {y_pred_test.min():.2f}, Max prédit: {y_pred_test.max():.2f}")
print(f"  - Min réel: {y_test_real.min():.2f}, Max réel: {y_test_real.max():.2f}")


### 7.2. Calcul des métriques d'évaluation

Calcul des métriques principales pour évaluer la performance du modèle : RMSE, MAE, et MAPE.


In [ ]:
# Calcul des métriques
rmse_test = np.sqrt(mean_squared_error(y_test_real, y_pred_test))
mae_test = mean_absolute_error(y_test_real, y_pred_test)
mape_test = mean_absolute_percentage_error(y_test_real, y_pred_test) * 100

# Affichage des résultats
print("=" * 50)
print("RÉSULTATS FINAUX - ÉVALUATION SUR LE TEST SET")
print("=" * 50)
print(f"RMSE Test : {rmse_test:.2f} (Root Mean Squared Error)")
print(f"MAE Test  : {mae_test:.2f} (Mean Absolute Error)")
print(f"MAPE Test : {mape_test:.2f}% (Mean Absolute Percentage Error)")
print("=" * 50)

# Visualisation des métriques
fig, ax = plt.subplots(figsize=(10, 6))
metrics = ['RMSE', 'MAE', 'MAPE (%)']
values = [rmse_test, mae_test, mape_test]
colors = ['steelblue', 'coral', 'teal']

bars = ax.bar(metrics, values, color=colors, alpha=0.7, edgecolor='black')
ax.set_title('Métriques d\'évaluation du modèle')
ax.set_ylabel('Valeur')
ax.grid(True, alpha=0.3, axis='y')

# Ajouter les valeurs sur les barres
for bar, val in zip(bars, values):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{val:.2f}', ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()


### 7.3. Visualisation des prédictions vs valeurs réelles

Comparaison visuelle des prédictions avec les valeurs réelles pour identifier les patterns d'erreur.


In [ ]:
# Graphique temporel : prédictions vs réelles (zoom sur les 200 dernières heures)
fig, axes = plt.subplots(2, 1, figsize=(15, 10))

# Zoom sur 200 heures
zoom_size = 200
axes[0].plot(y_test_real[-zoom_size:], label="Valeurs réelles", 
             color='black', alpha=0.7, linewidth=1.5)
axes[0].plot(y_pred_test[-zoom_size:], label=f"Prédictions (MAPE: {mape_test:.2f}%)", 
             color='red', linestyle='--', linewidth=1.5, alpha=0.8)
axes[0].set_title(f'Comparaison Prédictions vs Réelles - Zoom sur {zoom_size} dernières heures')
axes[0].set_xlabel('Heures (dans le test set)')
axes[0].set_ylabel('Consommation')
axes[0].legend(loc='best')
axes[0].grid(True, alpha=0.3)

# Erreurs résiduelles
errors = y_test_real[-zoom_size:] - y_pred_test[-zoom_size:]
axes[1].plot(errors, color='purple', alpha=0.7, linewidth=1)
axes[1].axhline(0, color='black', linestyle='-', linewidth=0.5)
axes[1].fill_between(range(len(errors)), errors, 0, alpha=0.3, color='purple')
axes[1].set_title(f'Erreurs résiduelles - Zoom sur {zoom_size} dernières heures')
axes[1].set_xlabel('Heures (dans le test set)')
axes[1].set_ylabel('Erreur (Réel - Prédit)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


### 7.4. Scatter plot et analyse de corrélation

Analyse de la corrélation entre prédictions et valeurs réelles pour évaluer la qualité du modèle.


In [ ]:
# Scatter plot : Prédictions vs Réelles
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Scatter plot avec ligne parfaite
correlation = np.corrcoef(y_test_real, y_pred_test)[0, 1]
axes[0].scatter(y_test_real, y_pred_test, alpha=0.5, color='blue', s=10)
axes[0].plot([y_test_real.min(), y_test_real.max()], 
             [y_test_real.min(), y_test_real.max()], 
             'r--', lw=2, label='Ligne parfaite (y=x)')
axes[0].set_xlabel('Valeurs Réelles')
axes[0].set_ylabel('Valeurs Prédites')
axes[0].set_title(f'Prédictions vs Réelles (Corrélation: {correlation:.3f}, MAPE: {mape_test:.2f}%)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Distribution des erreurs
errors_all = y_test_real - y_pred_test
axes[1].hist(errors_all, bins=50, edgecolor='black', alpha=0.7, color='coral')
axes[1].axvline(0, color='r', linestyle='--', linewidth=2, label='Erreur = 0')
axes[1].axvline(errors_all.mean(), color='g', linestyle='--', linewidth=2, 
                label=f'Moyenne: {errors_all.mean():.2f}')
axes[1].set_title('Distribution des erreurs de prédiction')
axes[1].set_xlabel('Erreur (Réel - Prédit)')
axes[1].set_ylabel('Fréquence')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Statistiques des erreurs
print(f"\nAnalyse des erreurs :")
print(f"  - Erreur moyenne: {errors_all.mean():.2f} (idéalement proche de 0)")
print(f"  - Écart-type des erreurs: {errors_all.std():.2f}")
print(f"  - Corrélation prédictions/réelles: {correlation:.4f}")
if abs(errors_all.mean()) < errors_all.std() * 0.1:
    print(f"  ✓ Le modèle ne présente pas de biais significatif")
else:
    print(f"  ⚠ Le modèle présente un biais (erreur moyenne ≠ 0)")


### 7.5. Analyse des erreurs par contexte

Analyse des performances selon différents contextes (heure, jour de la semaine, week-end) pour identifier les situations où le modèle performe moins bien.


In [ ]:
# Extraction des contextes pour le test set
test_heures = Heure_orig[168+split_index:]
test_jours_semaine = jours_de_la_semaine[168+split_index:]
test_is_weekend = is_weekend[168+split_index:]

# Calcul du MAPE par contexte
def calculate_mape_by_context(context_values, context_name):
    """Calcule le MAPE pour chaque valeur unique du contexte"""
    unique_values = np.unique(context_values)
    mape_by_context = {}
    for val in unique_values:
        mask = context_values == val
        if mask.sum() > 0:
            mape = mean_absolute_percentage_error(y_test_real[mask], y_pred_test[mask]) * 100
            mape_by_context[val] = mape
    return mape_by_context

# MAPE par heure
mape_by_hour = calculate_mape_by_context(test_heures, "heure")
mape_by_weekday = calculate_mape_by_context(test_jours_semaine, "jour_semaine")

# Visualisation
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# MAPE par heure
hours = sorted(mape_by_hour.keys())
mape_hours = [mape_by_hour[h] for h in hours]
axes[0].plot(hours, mape_hours, marker='o', linewidth=2, markersize=6, color='steelblue')
axes[0].axhline(mape_test, color='r', linestyle='--', label=f'MAPE moyen: {mape_test:.2f}%')
axes[0].set_title('MAPE par heure de la journée')
axes[0].set_xlabel('Heure')
axes[0].set_ylabel('MAPE (%)')
axes[0].set_xticks(range(0, 24, 2))
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# MAPE par jour de la semaine
jours_noms = ['Lun', 'Mar', 'Mer', 'Jeu', 'Ven', 'Sam', 'Dim']
jours_idx = sorted(mape_by_weekday.keys())
mape_days = [mape_by_weekday[j] for j in jours_idx]
axes[1].bar(jours_noms, mape_days, color='coral', alpha=0.7)
axes[1].axhline(mape_test, color='r', linestyle='--', label=f'MAPE moyen: {mape_test:.2f}%')
axes[1].set_title('MAPE par jour de la semaine')
axes[1].set_ylabel('MAPE (%)')
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# Identification des heures/jours avec les meilleures/pires performances
worst_hour = max(mape_by_hour, key=mape_by_hour.get)
best_hour = min(mape_by_hour, key=mape_by_hour.get)
worst_day = max(mape_by_weekday, key=mape_by_weekday.get)
best_day = min(mape_by_weekday, key=mape_by_weekday.get)

print(f"\nAnalyse des performances par contexte :")
print(f"  - Meilleure heure: {best_hour}h (MAPE: {mape_by_hour[best_hour]:.2f}%)")
print(f"  - Pire heure: {worst_hour}h (MAPE: {mape_by_hour[worst_hour]:.2f}%)")
print(f"  - Meilleur jour: {jours_noms[best_day]} (MAPE: {mape_by_weekday[best_day]:.2f}%)")
print(f"  - Pire jour: {jours_noms[worst_day]} (MAPE: {mape_by_weekday[worst_day]:.2f}%)")


## 8. Analyse des Résultats et Conclusion

### 📊 Interprétation & Pistes d'amélioration - Évaluation

**Observations clés :**
- **Performance globale** : Le MAPE, RMSE et MAE donnent une idée de la précision du modèle
- **Distribution des erreurs** : Si centrée sur 0, le modèle n'a pas de biais systématique
- **Erreurs par contexte** : Identifier les heures/jours où le modèle performe moins bien permet de cibler les améliorations

**Points forts identifiés :**
- Les lags temporels (surtout lag_24h) sont très prédictifs
- Les encodages cycliques capturent bien les patterns saisonniers
- Le modèle suit généralement bien les tendances

**Points à améliorer :**
- Si certaines heures/jours ont un MAPE élevé, créer des features spécifiques pour ces contextes
- Si les erreurs sont corrélées (autocorrélation), considérer des modèles de séries temporelles (LSTM, ARIMA)
- Tester des ensembles de modèles (stacking, blending) pour améliorer la robustesse

**Pistes d'amélioration techniques :**
1. **Feature engineering avancé** :
   - Features d'interaction (météo × saison, week-end × heure)
   - Moyennes mobiles et tendances
   - Features de pointe (détection des pics de consommation)

2. **Modélisation** :
   - Hyperparameter tuning (GridSearch/RandomSearch)
   - Modèles de séries temporelles (LSTM, GRU, Transformer)
   - Ensemble methods (XGBoost, LightGBM, stacking)

3. **Validation** :
   - Validation croisée temporelle (TimeSeriesSplit)
   - Validation sur plusieurs périodes pour tester la robustesse

4. **Post-processing** :
   - Lissage des prédictions si nécessaire
   - Ajustement des prédictions selon le contexte (calibrage)

---

**Notebook prêt pour itération et amélioration continue ! 🚀**
